In [42]:
import os
import xml.etree.ElementTree as ET
import kagglehub
from collections import Counter
import os
import pandas as pd
import json

# Download latest version
path = kagglehub.dataset_download("stanislavlevendeev/hazmat-detection")

print("Path to dataset files:", path)

100%|██████████| 14.9M/14.9M [00:03<00:00, 4.58MB/s]

Extracting files...


Path to dataset files: C:\Users\ewald\.cache\kagglehub\datasets\stanislavlevendeev\hazmat-detection\versions\3


In [43]:
file_path = os.path.join(path, 'cvat-for-video-dataset-hazmat-codes', 'annotations.xml')

=tree = ET.parse(file_path)
root = tree.getroot()

tasks = []
for task in root.findall(".//task"):
    task_id = task.find("id").text
    task_source = task.find("source").text
    tasks.append({"id": task_id, "source": task_source})

for task in tasks:
    print(f"Task ID: {task['id']}, source: {task['source']}")

print(len(tasks), "tasks found")

Task ID: 138, source: 1690279852.mp4
Task ID: 142, source: 1690281365.mp4
Task ID: 143, source: 1690279852.mp4
Task ID: 148, source: 1692830440.mp4
Task ID: 149, source: 1690801380.mp4
Task ID: 150, source: 1691487366.mp4
Task ID: 151, source: 1692787289.mp4
Task ID: 155, source: 1692875102.mp4
Task ID: 158, source: 1692945482.mp4
Task ID: 161, source: 1693811855.mp4
Task ID: 162, source: 1693954819.mp4
Task ID: 163, source: 1693805101.mp4
Task ID: 164, source: 1693820172.mp4
Task ID: 166, source: 1692787289.mp4
Task ID: 167, source: 1691496786.mp4
Task ID: 168, source: 1692872075.mp4
Task ID: 169, source: 1693308534.mp4
Task ID: 170, source: 1693308657.mp4
Task ID: 171, source: 1693309263.mp4
Task ID: 172, source: 1693820241.mp4
Task ID: 173, source: 1693820504.mp4
Task ID: 174, source: 1693820904.mp4
Task ID: 175, source: 1696009577.mp4
Task ID: 176, source: 1696374314.mp4
Task ID: 177, source: 1696416413.mp4
Task ID: 178, source: 1696441496.mp4
26 tasks found


In [44]:
# Directory containing video files
video_directory = os.environ["PATH_TO_DATA"]
print (video_directory)

data/videos


In [45]:
video_files = []
#  make a list of only the videos mentioned in the tasks
for task in tasks:
    video_name = task["source"]
    video_path = os.path.join(video_directory, video_name)
    if os.path.exists(video_path):
        video_files.append(video_path)
    else:
        print(f"Video file {video_path} not found")
print(f"{len(video_files)}/{len(tasks)} video files found")

Video file data/videos\1692830440.mp4 not found
Video file data/videos\1690801380.mp4 not found
Video file data/videos\1691487366.mp4 not found
Video file data/videos\1692787289.mp4 not found
Video file data/videos\1692945482.mp4 not found
Video file data/videos\1693811855.mp4 not found
Video file data/videos\1693954819.mp4 not found
Video file data/videos\1693805101.mp4 not found
Video file data/videos\1693820172.mp4 not found
Video file data/videos\1692787289.mp4 not found
Video file data/videos\1691496786.mp4 not found
Video file data/videos\1692872075.mp4 not found
Video file data/videos\1693308657.mp4 not found
Video file data/videos\1693309263.mp4 not found
Video file data/videos\1693820241.mp4 not found
Video file data/videos\1693820504.mp4 not found
Video file data/videos\1693820904.mp4 not found
Video file data/videos\1696009577.mp4 not found
Video file data/videos\1696374314.mp4 not found
Video file data/videos\1696416413.mp4 not found
Video file data/videos\1696441496.mp4 no

In [46]:
# find amount of frames with hazmat codes
frames_with_hazmat = 0
codes = []
root = tree.getroot()
for track in root:
    for box in track.findall("box"):
        for attribute in box.findall("attribute"):
            if attribute.get("name") == "code":

                code_value = attribute.text
                frames_with_hazmat += 1
                codes.append(code_value)

print(f"Frames with hazmat codes: {frames_with_hazmat}")
# print value keypairs of codes
counter = Counter(codes)
diff_codes = len(counter)
for item, count in counter.items():
    print(f"{item}: {count}")

print(f"Amount of different codes: {diff_codes} ")


Frames with hazmat codes: 10734
83/2789: 237
23/1055: 177
83/1715: 136
23/1969: 157
263/1040: 2068
39/2055: 3016
668/2927: 2121
58/2014: 662
63/2023: 287
283/1040: 52
33/1280: 202
263/2040: 75
839/2218: 493
68/2076: 120
80/2215: 126
90/3077: 760
30/2055: 45
Amount of different codes: 17


In [47]:
data_dir = os.environ["PATH_TO_LABELS_JSON"]
files = [f for f in os.listdir(data_dir) if f.endswith('.json')]

data_list = []

for file in files:
    file_path = os.path.join(data_dir, file)
    
    with open(file_path, 'r') as f:
        data = json.load(f)
        data_list.append(data)

df = pd.DataFrame(data_list)

print(df)


                                            description number class
0     Ammonium picrate, dry or wetted with less than...   0004     1
1          Cartridges for weapons, with bursting charge   0005     1
2          Cartridges for weapons, with bursting charge   0006     1
3          Cartridges for weapons, with bursting charge   0007     1
4     Ammunition, incendiary with or without burster...   0009     1
...                                                 ...    ...   ...
2346                    Batteries, nickel-metal hydride   3496     9
2347                                         Krill meal   3497   4.2
2348                        Iodine Monochloride, liquid   3498     8
2349  Capacitor, electric double layer (with an ener...   3499     9
2350                   Chemical under Pressure, N.O.S.}   3500   2.2

[2351 rows x 3 columns]


In [4]:
df.head()

,description,number,class
0,"Ammonium picrate, dry or wetted with less than...",0004,1
1,"Cartridges for weapons, with bursting charge",0005,1
2,"Cartridges for weapons, with bursting charge",0006,1
3,"Cartridges for weapons, with bursting charge",0007,1
4,"Ammunition, incendiary with or without burster...",0009,1


In [7]:
df.to_csv('./data/un-number-labels.csv', index=False)

In [40]:
for code in codes:
    code_x = None
    if code is not None:
        code_x = code.split("/")[-1]
    if not df[df['number'] == code_x].empty:
        code_label = df[df['number'] == code_x]['description'].values[0]
    print(f"{code_x}: {code_label}")


2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent acid, by mass
2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent acid, by mass
2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent acid, by mass
2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent acid, by mass
2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent acid, by mass
2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent acid, by mass
2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent acid, by mass
2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent acid, by mass
2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent acid, by mass
2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent acid, by mass
2789: Acetic acid, glacial or Acetic acid solution, with more than 80 percent ac

In [48]:
print(f"Amount of different codes: {diff_codes} out of {len(df)}")


Amount of different codes: 17 out of 2351
